<a href="https://colab.research.google.com/github/ownerofglory/hillel-expense-tracker-backend/blob/feature%2F79-gpt-sample-with-jupyter-noterbook/Hillel_Expense_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hillel Expense Tracker
### Using GPT 4o-mini

In [8]:
import os
from openai import OpenAI

In [9]:
from google.colab import userdata

In [10]:
import base64

In [11]:
import json

In [12]:
client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY')
)

## Bill scanning example

### Load image and convert it into base64

In [14]:
with open("sample_data/supermarket1.jpg", "rb") as image_file:
    img_base64 = base64.b64encode(image_file.read()).decode('utf-8')

### Create bill scanning context

In [15]:
context = """
 You are a very polite AI agent of "Hillel Tracker App",
 built by students and a teacher of "Hillel IT School" within Java Pro course.
 Your task is to parse supermarket bill and extract data about spent money.
 Consider that there are following categories for expenses:
[Food and drinks,Utilities,Entertainment,Transport,Health and fitness,Education,Shopping,Others]
"""

### Define bill scan response schema

In [16]:
bill_scan_schema = {
        "type": "json_schema",
        "json_schema": {
            "name": "bill_scan",
            "strict": True,
            "schema": {
                "type": "object",
                "properties": {
                    "amount": {
                        "type": "number",
                        "description": "The total amount of the payment."
                    },
                    "currency": {
                        "type": "string",
                        "description": "The currency in which the payment is made."
                    },
                    "category": {
                        "type": "string",
                        "description": "Category assigned to the payment."
                    },
                    "description": {
                        "type": "string",
                        "description": "Description of the payment."
                    }
                },
                "required": [
                    "amount",
                    "currency",
                    "category",
                    "description"
                ],
                "additionalProperties": False
            }
        }
    }

### Call chat completion API



In [17]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": context
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": ""},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                },
            ],
        }
    ],
    response_format=bill_scan_schema
)

In [18]:
json_object = json.loads(response.choices[0].message.content)

json_formatted_str = json.dumps(json_object, indent=2)

print(json_formatted_str)

{
  "amount": 100,
  "currency": "RJ",
  "category": "Food and drinks",
  "description": "Supermarket bill including produce and deli items."
}


## Expense categorizing example

### Create categorizing context

In [19]:
context = """
You are a very polite AI agent of "Hillel Tracker App",
built by students and a teacher of "Hillel IT School" within Java Pro course.
Your task is to parse supermarket bill and extract data about spent money.
Consider that there are following categories for expenses:
[Food and drinks,Utilities,Entertainment,Transport,Health and fitness,Education,Shopping,Others].
Default currency is EUR
"""

### Define categorizing response schema

In [20]:
categorizing_schema = {
        "type": "json_schema",
        "json_schema": {
            "name": "categorizing",
            "strict": True,
            "schema": {
                "type": "object",
                "properties": {
                    "amount": {
                        "type": "number",
                        "description": "The total amount of the payment."
                    },
                    "currency": {
                        "type": "string",
                        "description": "The currency in which the payment is made."
                    },
                    "category": {
                        "type": "string",
                        "description": "Category assigned to the payment."
                    },
                    "description": {
                        "type": "string",
                        "description": "Description of the payment."
                    }
                },
                "required": [
                    "amount",
                    "currency",
                    "category",
                    "description"
                ],
                "additionalProperties": False
            }
        }
    }

### Call chat completion API

In [21]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": context
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "I went to a japanease restaurant and spent there 50 bucks"}
            ],
        }
    ],
    response_format=categorizing_schema
)

In [22]:
json_object = json.loads(response.choices[0].message.content)

json_formatted_str = json.dumps(json_object, indent=2)

print(json_formatted_str)

{
  "amount": 50,
  "currency": "EUR",
  "category": "Food and drinks",
  "description": "Dinner at a Japanese restaurant"
}
